Exercicis d'Hipotesis Testing.

EX01. Descarrega el dataset adjunt de dades oficials de la UEFA i selecciona un atribut del conjunt de dades. Calcula el p-valor i digues si rebutja la hipòtesi nul·la agafant un alfa de 5%.

In [62]:
import pandas as pd

UEFA = pd.read_csv('Lligues europees UEFA.csv', sep=';', encoding='latin1')

In [91]:
print(str(UEFA.columns)+'\n')
print(UEFA.sample(5))

Index(['Rk', 'Squad', 'Country', 'LgRk', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD',
       'Pts', 'Pts/G', 'xG', 'xGA', 'xGD', 'xGD/90', 'Last 5', 'Attendance',
       'Top Team Scorer', 'Goalkeeper'],
      dtype='object')

    Rk       Squad Country  LgRk  MP   W   D   L  GF  GA  ...  Pts  Pts/G  \
28  29  Fiorentina     ITA     7  36  18   5  13  56  47  ...   59   1.64   
1    2   Liverpool     ENG     2  36  26   8   2  89  24  ...   86   2.39   
38  39      Nantes     FRA     9  37  15   9  13  54  47  ...   54   1.46   
15  16   Marseille     FRA     3  37  20   8   9  59  38  ...   68   1.84   
85  86     Levante     ESP    19  37   7  11  19  47  74  ...   32   0.86   

      xG   xGA   xGD  xGD/90     Last 5 Attendance         Top Team Scorer  \
28  55.4  42.4  12.9    0.36  W L L L W      19801     Duan Vlahovi? - 17   
1   84.6  33.1  51.4    1.43  W W W D W      53367      Mohamed Salah - 22   
38  41.7  53.8 -12.1   -0.33  L W D W L      20368  Randal Kolo Muani - 12   
15  

Vemos todas las columnas para elegir la que más nos interesa. Nos quedaremos con puntos por partido, que parece la más interesante / objetiva, de cada a ver el rendimiento real de los equipos.

In [92]:
goleadores = UEFA['Top Team Scorer']
goleadores = [x.split(' - ') for x in goleadores]
goles = [x[1] for x in goleadores]
n_goles = [int(goles) for goles in goles]
print(n_goles)

[15, 22, 27, 35, 25, 11, 21, 14, 22, 12, 11, 10, 24, 24, 21, 12, 11, 11, 9, 21, 16, 27, 20, 12, 13, 15, 9, 12, 17, 15, 9, 13, 20, 18, 20, 10, 12, 8, 12, 15, 16, 6, 8, 11, 16, 8, 14, 12, 8, 11, 8, 13, 17, 9, 8, 12, 8, 10, 11, 11, 12, 6, 10, 14, 7, 9, 17, 9, 15, 8, 8, 13, 5, 10, 9, 7, 6, 8, 5, 13, 6, 8, 10, 8, 11, 13, 5, 14, 10, 10, 8, 13, 9, 11, 9, 10, 11, 9]


Ahora ya tenemos la muestra de goles de los máximos goleadores de los equipos punteros de las ligas europeas en una lista, para poder operar con ella.

In [93]:
from scipy.stats import shapiro

stat, p = shapiro(n_goles)
print('stat=%.2f, p=%.2f\n' % (stat, p))
if p > 0.05:
 print('Gaussiana')
else:
 print('No gaussiana')

stat=0.88, p=0.00

No gaussiana


Shapiro mide la hipótesis nula de que la distribución sea Gaussiana en base al p valor. En este caso, la probabilidad de encontrar unos datos como los de `Top Team Scorer` si la distribución fuera Gaussiana es de 0,00 (p-valor), que está por debajo de la alpha de 5%, por lo tanto rechazamos la hipótesis nula - que siga una distribución normal - con un nivel de confianza del 95%.

EX02. Amb el mateix dataset selecciona dos altres atributs del conjunt de dades. Calcula els p-valors i digues si rebutgen la hipòtesi nul·la agafant un alfa de 5%.

Bien, ahora vamos a comparar datos de distintas ligas europeas, lo que puede resultar más interesante, quedándonos solamente con las muestras correspondientes a cada país. Elegiremos para empezar los goles en contra, como indicador de juego defensivo, comparando Italia con España.

In [94]:
print(UEFA['Country'].value_counts())

ENG    20
ESP    20
FRA    20
ITA    20
GER    18
Name: Country, dtype: int64


Comprobamos el tamaño de la muestra, en el caso de Italia y España, que es 20.

In [95]:
print('Italia: '+str(UEFA[(UEFA['Country'] == 'ITA')]['GA'].mean()))
print('Italia: '+str(UEFA[(UEFA['Country'] == 'ESP')]['GA'].mean()))


Italia: 52.7
Italia: 46.5


Imprimimos las respectivas medias para tener una idea general. Vemos como la media de goles en contra de los equipos punteros en Italia no sólo no es menor, sino que es mayor que en España. En cualquier caso vamos a hacer nuestro test.

In [96]:
from scipy.stats import ttest_ind

GA_ESP = UEFA[(UEFA['Country'] == 'ESP')]['GA']
GA_ITA = UEFA[(UEFA['Country'] == 'ITA')]['GA']

stat, p = ttest_ind(GA_ITA, GA_ESP, alternative='less')
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
 print('La media de goles en contra en Italia es igual o mayor a la de España')
else:
 print('La media de goles en contra en Italia es probablemente inferior a la de España')

stat=1.480, p=0.926
La media de goles en contra en Italia es igual o mayor a la de España


Realizamos el test t-student para falsar la hipótesis nula de que Italia tiene una media de goles en contra mayor o igual que la de España. Las muestras nos indican que no podemos rechazar la hipótesis nula, pues la probabilidad de que sea así es del 92,6%, muy por encima del 5% a partir del cual podríamos pensar en rechazar la hipótesis nula con un nivel de confianza del 95%.

A continuación nos centraremos de nuevo en la media de puntos por partido de los equipos punteros, comparando España con Reino Unido (ENG), como indicador de si una liga es más fácil que la otra.

In [48]:
print(round(UEFA[(UEFA['Country'] == 'ESP')]['Pts/G'].mean(), 2))
print(round(UEFA[(UEFA['Country'] == 'ENG')]['Pts/G'].mean(), 2))

1.35
1.38


Vemos las medias de España e Inglaterra, respectivamente.

In [50]:
PG_ESP = UEFA[(UEFA['Country'] == 'ESP')]['Pts/G']
PG_ENG = UEFA[(UEFA['Country'] == 'ENG')]['Pts/G']

Guardamos las muestras en dos variables.

In [56]:
stat, p = ttest_ind(PG_ESP, PG_ENG)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
 print('No hay una liga que sea más fácil que la otra entre la Premier League inglesa y la Liga española.')
else:
 print('La Premier League inglesa y la Liga española no son igual de fáciles')

stat=-0.199, p=0.843
No hay una liga que sea más fácil que la otra entre la Premier League inglesa y la Liga española.


Para finalmente aplicar el T-Test, que nos permite descartar que haya una liga más fácil que la otra con los datos de las muestras respectivas de puntos por partido. El p-valor está en 84,3%, muy por encima del 5% a partir del que rechazaríamos la hipótesis nula.

EX03. Continua amb el conjunt de dades adjunt i selecciona tres atributs del conjunt de dades. Calcula el p-valor i digues si rebutja la hipòtesi nul·la agafant un alfa de 5%.

Por último, nos piden comparar tres medidas. Vamos con el juego ofensivo, compararemos los goles a favor en Reino Unido, Italia y España.

In [57]:
print(UEFA[(UEFA['Country'] == 'ESP')]['GF'].mean())
print(UEFA[(UEFA['Country'] == 'ENG')]['GF'].mean())
print(UEFA[(UEFA['Country'] == 'ITA')]['GF'].mean())

46.5
50.9
52.7


Comenzamos visualizando la media de goles a favor de los 20 equipos punteros de cada uno de los países, España, Inglaterra e Italia, en este orden.

In [59]:
GF_ESP = UEFA[(UEFA['Country'] == 'ESP')]['GF']
GF_ENG = UEFA[(UEFA['Country'] == 'ENG')]['GF']
GF_ITA = UEFA[(UEFA['Country'] == 'ITA')]['GF']

Guardamos las tres muestras en su variable correspondiente.

In [97]:
from scipy.stats import f_oneway

stat, p = f_oneway(GF_ESP, GF_ENG, GF_ITA)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
 print('Las medias de goles a favor no presenta una diferencia significativa entre las tres ligas de España, Inglaterra e Italia.')
else:
 print('Alguna de las ligas entre España, Inglaterra e Italia, es significativamente más ofensiva que las otras.')

stat=0.807, p=0.451
Las medias de goles a favor no presenta una diferencia significativa entre las tres ligas de España, Inglaterra e Italia.


Aplicamos el test ANOVA para comparar las tres medias de goles a favor. El p-valor es de 0.451, es decir, también muy superior al 0.05 que nos permitiría descartar la hipótesis nula de que las medias son iguales. Por lo tanto concluímos que no hay una diferencia significativa en el nivel de dificultad de las ligas.

Finalmente, como resumen, primero hemos utilizado un test de Shapiro-Wilk para comprobar la normalidad de los datos referentes a los máximos goleadores de las ligas europeas, descartando la hipótesis de que sigan una distribución gaussiana - normal. En segundo lugar hemos comparado los dos 'atributos' de goles en contra para los 20 principales equipos de las ligas italiana y español con un test T, no pudiendo afirmar que la media de goles en contra en Italia sea significativamente menor que en España. Por último hemos utilizado un test ANOVA para comparar las medias de puntos por partido de los 20 principales equipos de las ligas española, inglesa e italiana, y tampoco hemos podido descartar que el nivel de dificultad sea parecido. En todos los casos el p-valor ha estado muy distante del necesario para concluir lo contrario.  
Destacar por sorprendente el primer caso, obteniendo una probabilidad del 0.00 para afirmar que los goles de los máximos artilleros de estas ligas no siguen una distribución normal.